# WildObs Tech Demo: Using YoloV11 for Australian Animal Detection

WildObs is a new national platform that will provide and end-to-end solution for wildlife monitoring in Australia. One of its key features is the ability to automatically detect Australian animals in images. For this, the WildObs team is developing the next generation of Australia's animal detection models using existing national and international tools.

In this notebook, we use the powerful image hosting services of the Atlas of Living Australia (ALA) and the new YoloV11 model to demonstrate how to use a base model to detect Australian animals in images. Let's see how good is this model at detecting an iconic Australian animal, the echidna

## Install Required Libraries
For this demo we will need the following libraries:
- `ultralytics` for the YoloV11 model
- `pandas` for data manipulation
- `pathlib` for file manipulation
- `galah` for the ALA image hosting services

You can install them using the following command:
```bash
!pip install -q ultralytics pandas pathlib galah
```


# Load the libraries

In [42]:
import os
from pathlib import Path
from ultralytics import YOLO 
import pandas as pd
import galah

## Define our variables

In [43]:
# Define the path to the folder containing the images
image_folder = 'media' 

# Load the YOLOv11 model
model = YOLO('yolo11n.pt')  


## Extract the class names from the base model


In [44]:
# extract class names from the model
class_names = model.names

# Create a dataframe from the class names
df = pd.DataFrame(list(class_names.items()), columns=['class_number', 'class_name'])

In [ ]:
model.predict(source=image_folder, save=True, save_txt=True)  # Predict the images in the folder

In [56]:


labels_dir = 'runs/detect/predict/labels'
all_data = []

# Iterate through each file in the labels directory
for label_file in os.listdir(labels_dir):
    # Check if the file is a .txt file
    if label_file.endswith(".txt"):
        # Read the contents of the label file
        with open(os.path.join(labels_dir, label_file), "r") as file:
            lines = file.readlines()
            for line in lines:
                values = line.split()
                # Add the label file name (without the .txt extension) as the image name
                image_name = label_file.replace('.txt', '')
                values.append(image_name)
                all_data.append(values)

# Create a DataFrame from the combined list, including the image_name column
final_df = pd.DataFrame(all_data, columns=["class_number", "cx", "cy", "width", "height", "image_name"])

# Filter the final DataFrame to include only the class number and the image name
filtered_df = final_df[['class_number', 'image_name']]

# Print the filtered DataFrame
print(filtered_df)


   class_number                            image_name
0            14  1ce6444e-e6f7-43bf-8ca3-a7468ffc04d2
1            14  0e9d776a-e758-4556-89a3-ec4bd49ece39
2            72  1c1571a0-df05-41e5-a3e8-43d534733af8
3            25  00b05da7-bfe7-4a6c-9c4c-ebe904e061b6
4            16  0d151fac-5f34-4252-81a7-f24a926cfa26
5            47  0f4ba498-a38b-4d4d-9fda-a8c21c402f6c
6             0  0ff79320-f691-4e52-9e35-583deae8e556
7            21  00d23640-f155-4aed-8379-b0bf6cf904ac
8            21  0a4c2e76-883c-45e6-8f26-1c6dd41d88e0
9            14  1dbc69c2-91bc-4780-a37e-51d361eb9edd
10           21  0a498cce-9acf-49fd-80bc-c31707155105


In [ ]:
# Convert class_number to string in both DataFrames to ensure they match
final_df['class_number'] = final_df['class_number'].astype(str)
df['class_number'] = df['class_number'].astype(str)

# Perform the merge after ensuring the same data type
final_df2 = final_df.merge(df, on='class_number', how='left')

# Print the resulting DataFrame
print(final_df2)